In [1]:
### get raw data path from json header, then load all the raw data
!pip install pandas
from datetime import datetime
import pandas as pd
import json
settings = json.load(open('settings.json'))
settings
# read csv
item_categories = pd.read_csv(settings["DATA_DIR"] + 'item_categories.csv')
items = pd.read_csv(settings["DATA_DIR"] + 'items.csv')
shops = pd.read_csv(settings["DATA_DIR"] + 'shops.csv')
sales_train = pd.read_csv(settings["TRAIN_DATA_PATH"],
                          parse_dates=['date'],
                          date_parser=lambda x: datetime.strptime(x, '%d.%m.%Y'))
test = pd.read_csv(settings["TEST_DATA_PATH"])
# $pip install nltk
# $pip install googletrans
# please run in terminals if you don't have them
# below are basics data processing/visualization

You should consider upgrading via the 'C:\Users\Ivan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# below are housekeeping, timing ,etc kits
import pickle
import gc
import re
import string 
import tqdm
import os

# below are modeling 
import sklearn
import lightgbm as lgb
import catboost

# below are NLP packages for text and google translate to translate russian into English
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
import googletrans
import re

# Below are the specific libraries in the packages
from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm,tqdm_notebook
from matplotlib import pyplot
from googletrans import Translator
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score
from sklearn.model_selection import KFold, train_test_split #StratifiedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.decomposition import NMF #PCA, TruncatedSVD, 
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
#from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic

from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#from nltk.stem import LancasterStemmer   # DO NOT USE- not reliable in this application
#stemmer= LancasterStemmer()
from itertools import tee, islice
from collections import Counter
from googletrans import Translator

for p in [np, pd, sklearn, lgb]:
    print (p.__name__, p.__version__)

# downcast variables to save memory
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

# get most frequent words
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    # https://gist.github.com/CristhianBoujon/c719ba2287a630a6d3821d37a9608ac8
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# get ngrams
def ngrams(lst, n):
  tlst = lst
  while True:
    a, b = tee(tlst)
    l = tuple(islice(a, n))
    if len(l) == n:
      yield l
      next(b)
      tlst = b
    else:
        if len(l)==1:
          yield lst
          tlst = lst
          break
        else:
          break
 # from https://stackoverflow.com/questions/12488722/counting-bigrams-pair-of-two-words-in-a-file-using-python
 # with modifications to trap errors in case sentence has only 1 word

def duplicate_removal(w):
    word_list = []
    [word_list.append(s) for s in w if s not in word_list]
    return word_list
#https://www.engineeringbigdata.com/python-exercise-10-remove-duplicate-words-with-user-input/   this removes duplicated words in a sentence

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


numpy 1.21.2
pandas 1.3.3
sklearn 1.0
lightgbm 3.3.0


In [15]:
# Setting up text - to - numeric - feature extraction code
# These are the routine customized methods for processing . here since categorical features for shop_name and category_name were less than 100,
# one can run these without much performance issues. I do not recommend using this for item_names as it has 22170 unique objects
tokenizer = MWETokenizer()
tokenizer.add_mwe(('xbox','one'))
tokenizer.add_mwe(('xbox','360'))
unwantedtokens=[':',',','(',')','-','.','!','/']
translator = Translator()
# google translate from Russian to English

def get_ngrams(text,n):
    n_grams = ngrams(tokenizer.tokenize((text).split()), n)
    return [ ' '.join(grams) for grams in n_grams]
    
def text_cleaning(text_df):
    # process the text in this mini routine
	# step1: translate using google translate
	# step2: lemmatize , gaming-game, xbox 360 with underscores
	# step3: remove dash, replace by underscores for two-words; then tokenize
	# step4: stem words (optional)
    for sentence in range(text_df.index.stop):
        temp=translator.translate(text_df[sentence],dest='en')
        testb=temp.text
        del temp
        # change gaming to game, blue-ray to blue_ray(to make sure it detect it as one word)
        temp=testb.replace('ing', 'e') 
        testb=temp
        del temp
        ngram2=get_ngrams(testb,2)
        # make xbox_360 and x_box_one etc
        testb=(" ").join(ngram2)   
        # remove tokens, then remove duplicates, lemmatize and stem (stem is not implemented today)
        text_tokens = word_tokenize(testb)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        filtered_sentence = [word for word in tokens_without_sw if not word in unwantedtokens]
        clean_sentence=" ".join(duplicate_removal(filtered_sentence))
        word_list = nltk.word_tokenize(clean_sentence)
        lemmatized_sentence = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        temp=lemmatized_sentence.replace('-', '_')  
        # change blue-ray to blue_ray to make sure it is treated as one word; since the stand-alone '-' has been removed, in-words '-' should be converted to '_'
        stemmed_sentence=temp
        del temp
        #stemmed_sentence= stemmer.stem(lemmatized_sentence)
        testa[sentence]=stemmed_sentence
    return text_df

def plot_top_words(text_df,num_top_words):
    common_words = get_top_n_words(text_df, num_top_words)
    # picked top 40 words, because 40 will start to have common_words with minimum frequency of 1 - trivial solution
    # you can modify it and pass it as argument instead
    for word, freq in common_words:
        print(word, freq)
    # for 'top words'. 
    keywords,frequency=zip(*common_words)
    fig, ax = plt.subplots()
    fig.set_size_inches(9,9)
    plt.title('keywords and frequency of appearance ')
    y_pos = np.arange(len(keywords))
    plt.barh(y_pos, frequency)
    plt.yticks(y_pos, keywords)
    plt.show()
    return

def text_tfidf_num(text_df):
    countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
    tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
    count_wm = countvectorizer.fit_transform(text_df)
    tfidf_wm = tfidfvectorizer.fit_transform(text_df)
    count_tokens = countvectorizer.get_feature_names()
    tfidf_tokens = tfidfvectorizer.get_feature_names()
    df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
    df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)
    print("Count Vectorizer\n")
    print(df_countvect.head(5))
    print("\nTD-IDF Vectorizer\n")
    print(df_tfidfvect.head(5))
    return df_tfidfvect

In [16]:
testa=item_categories.item_category_name
testb=text_cleaning(testa)
plot_top_words(testb,40)
item_cat_num=text_tfidf_num(testb)

Count Vectorizer

   1c  360  3d  4k  accessories  additional  albums  android  artbooks  \
0   0    0   0   0            0           0       0        0         0   
1   0    0   0   0            1           0       0        0         0   
2   0    0   0   0            1           0       0        0         0   
3   0    0   0   0            1           0       0        0         0   
4   0    0   0   0            1           0       0        0         0   

   attribute  ...  standard  stickers  tickets  toys  traine  video  vinyl  \
0          0  ...         0         0        0     0       0      0      0   
1          0  ...         0         0        0     0       0      0      0   
2          0  ...         0         0        0     0       0      0      0   
3          0  ...         0         0        0     0       0      0      0   
4          0  ...         0         0        0     0       0      0      0   

   windows  xbox  игры  
0        0     0     0  
1        0     0  

C:\Users\Ivan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
item_category_name.info()

NameError: name 'item_category_name' is not defined